## Web Scraping

Este codigo es utilizado para realizar el web scraping de las ofertas de empleo relacionado a las de Data. Se recomienda realizarlas semanalmente para mantener el flujo del dato constante sin obtener muchos duplicados.
El web scraping realizado nos trae los datos mas relevantes en este caso de "AI Jobs" el cual se guarda en un dataset y posteriormente se exporta en un archivo csv llamado "ai_jobs.csv" para su posterior transformación, el cual esta descrito con todo y codigo en el archivo "transform.ipynb"

**Importante**: Con este codigo se obtienen las 95 ofertas laborales mas recientes

## Aqui comienza el codigo

In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [36]:
BASE_URL = 'https://ai-jobs.net'
JOBS_URL = 'https://ai-jobs.net/?cat=12&cat=10&cat=3&cat=6&cat=2&cat=5&cat=15&cat=16&key=&exp='

In [38]:
def extract_job_info(job_element):
    job_info = {}
    
    try:
        job_info['title'] = job_element.find('h3', {'class': 'h5 mb-2 text-body-emphasis'}).get_text()
    except AttributeError:
        job_info['title'] = "No disponible"
    
    # Agrega aquí más campos usando la misma estructura try-except.

    try:
        job_info['level'] = job_element.find('span', {'class': 'badge rounded-pill text-bg-info my-md-1 d-none d-md-inline-block'}).get_text()
    except AttributeError:
        job_info['level'] = "No disponible"

    try:
        job_info['job_type'] = job_element.find('span', {'class': 'badge rounded-pill text-bg-secondary my-md-1 ms-1'}).get_text()
    except AttributeError:
        job_info['job_type'] = "No disponible"

    try:
        job_info['salary_range'] = job_element.find('span', {'class': 'badge rounded-pill text-bg-success d-none d-md-inline-block'}).get_text()
    except AttributeError:
        job_info['salary_range'] = "No disponible"

    try:
        job_info['company_name'] = job_element.find('p', {'class': 'm-0 text-muted'}).get_text()
    except AttributeError:
        job_info['company_name'] = "No disponible"

    try:
        job_info['location_name'] = job_element.find('span', {'class': 'd-none d-md-block text-break'}).get_text()
    except AttributeError:
        job_info['location_name'] = "No disponible"

    # Extraemos las skills y benefict en una lista con un ciclo for
    skills = []
    try:
        skill_elements = job_element.find_all('span', {'class': 'badge rounded-pill text-bg-light'})
        for skill_element in skill_elements:
            skills.append(skill_element.get_text())
    except AttributeError:
        pass
    job_info['skills'] = skills

    benefits = []
    try:
        benefits_elements = job_element.find_all('span', {'class': 'badge rounded-pill text-bg-success'})
        for benefit_element in benefits_elements:
            benefits.append(benefit_element.get_text())
    except AttributeError:
        pass
    job_info['benefits'] = benefits

    
    return job_info

In [39]:
def get_total_pages(soup):
    pagination = soup.find('ul', {'class': 'pagination'})
    if pagination:
        return int(pagination.find_all('li')[-2].get_text())
    else:
        return 1

In [40]:
all_jobs = []

try:
    response = requests.get(JOBS_URL)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')
    
    total_pages = get_total_pages(soup)
    
    for page in range(1, total_pages + 1):
        if page > 1:
            response = requests.get(f"{JOBS_URL}&paged={page}")
            soup = BeautifulSoup(response.content, 'html.parser')
        
        job_elements = soup.find('ul', {'class':'list-group list-group-flush mb-4'}).find_all('li', {'class': 'list-group-item list-group-item-action p-1'})
        
        for job_element in job_elements:
            job_info = extract_job_info(job_element)
            all_jobs.append(job_info)
    
    print(f"Se han extraído {len(all_jobs)} ofertas de trabajo.")
    
except requests.exceptions.HTTPError as err:
    print(f'Error: {err}')
except requests.exceptions.RequestException as e:
    print(f'Error de conexión: {e}')

print(all_jobs)

'''
La función extract_job_info() se utiliza para extraer la información de una oferta de trabajo y manejar datos faltantes.
La función get_total_pages() se utiliza para determinar el número total de páginas de ofertas de trabajo.
El código ahora itera a través de todas las páginas de ofertas de trabajo y extrae la información utilizando la función extract_job_info().
Se almacena la información extraída en una lista de diccionarios llamada all_jobs.
Este código mejorado debería ayudarte a hacer web scraping a toda la página del sitio web y manejar mejor los datos faltantes. 
No olvides agregar los campos adicionales que necesitas en la función extract_job_info() siguiendo la misma estructura try-except.
'''

Se han extraído 95 ofertas de trabajo.
[{'title': 'Remote AI/ML Engineer in Wildlife Conservation', 'level': 'Mid-level', 'job_type': 'Contract', 'salary_range': 'USD 25K - 30K', 'company_name': 'The Wild Neighbors Database Project', 'location_name': 'Remote, US', 'skills': ['APIs', 'Classification', 'Data management', 'GitHub', 'Keras', 'Machine Learning', 'ML models', '+6'], 'benefits': ['Career development', 'Health care']}, {'title': 'AI/ML-powered Trading Strategy Researcher (and Implementer)', 'level': 'Senior-level', 'job_type': 'Full Time', 'salary_range': 'USD 100K - 200K', 'company_name': 'White Wind Research', 'location_name': 'Remote', 'skills': ['Finance', 'Machine Learning', 'Python', 'Research', 'Trading Strategies'], 'benefits': []}, {'title': 'Principal Data Engineer', 'level': 'Senior-level', 'job_type': 'Full Time', 'salary_range': 'USD 175K - 225K', 'company_name': 'Murmuration', 'location_name': 'Remote, US', 'skills': ['Agile', 'Architecture', 'Athena', 'AWS', 'Bi

'\nEste código sigue la estructura general del código original, pero se han realizado mejoras:\n\nLa función extract_job_info() se utiliza para extraer la información de una oferta de trabajo y manejar datos faltantes.\nLa función get_total_pages() se utiliza para determinar el número total de páginas de ofertas de trabajo.\nEl código ahora itera a través de todas las páginas de ofertas de trabajo y extrae la información utilizando la función extract_job_info().\nSe almacena la información extraída en una lista de diccionarios llamada all_jobs.\nEste código mejorado debería ayudarte a hacer web scraping a toda la página del sitio web y manejar mejor los datos faltantes. \nNo olvides agregar los campos adicionales que necesitas en la función extract_job_info() siguiendo la misma estructura try-except.\n'

In [41]:
# Convertir la lista de diccionarios all_jobs en un DataFrame de Pandas
jobs_df = pd.DataFrame(all_jobs)

In [42]:
# Le damos un formato mas legible y convertimos las listas de skills y benefits en cadenas separadas por comas
jobs_df['skills'] = jobs_df['skills'].apply(lambda x: ', '.join(x))
jobs_df['benefits'] = jobs_df['benefits'].apply(lambda x: ', '.join(x))

#jobs_df.to_csv('ai_jobs.csv', index=False)

In [43]:
# Mostrar las primeras filas del DataFrame
jobs_df

,title,level,job_type,salary_range,company_name,location_name,skills,benefits
0,Remote AI/ML Engineer in Wildlife Conservation,Mid-level,Contract,USD 25K - 30K,The Wild Neighbors Database Project,"Remote, US","APIs, Classification, Data management, GitHub,...","Career development, Health care"
1,AI/ML-powered Trading Strategy Researcher (and...,Senior-level,Full Time,USD 100K - 200K,White Wind Research,Remote,"Finance, Machine Learning, Python, Research, T...",
2,Principal Data Engineer,Senior-level,Full Time,USD 175K - 225K,Murmuration,"Remote, US","Agile, Architecture, Athena, AWS, Big Data, CI...",Equity
3,MLOps Engineer,Mid-level,Full Time,USD 124K - 134K,Hala Systems,"Remote, EU and US","Architecture, CI/CD, Computer Science, Compute...","Career development, Competitive pay, Flex hour..."
4,AI Developer,Senior-level,Full Time,USD 60K - 108K,Lemon.io,Ukraine,"Caffe, Computer Science, Computer Vision, Deep...","Career development, Flex hours, Startup enviro..."
...,...,...,...,...,...,...,...,...
90,Technical Project Manager AI/ML,Executive-level,Full Time,USD 65K - 121K *,Bosch Group,"Bengaluru, India","Agile, Computer Vision, Engineering, Machine L...","Career development, Team events"
91,"Sr. Staff Computer Vision Engineer, Robotics",Senior-level,Full Time,USD 184K - 342K,Coupang,"Mountain View, USA","Agile, Computer Science, Computer Vision, E-co...","401(k) matching, Career development, Equity, F..."
92,UX Data Analyst,Senior-level,Full Time,USD 92K - 130K *,Adyen,Amsterdam,"A/B testing, Data analysis, Data strategy, Dat...","Career development, Flex vacation, Relocation ..."
93,Senior Autonomy Research Engineer,Senior-level,Full Time,USD 150K - 249K *,STR,"Woburn, Massachusetts, United States","CI/CD, Computer Science, DevOps, Distributed S...",Career development


In [44]:
# Exportamos el la data extraida a un formato csv
#jobs_df.to_csv('ai_jobs.csv', index=False)